### Dziuk的平均曲率流演化

In [1]:
from ngsolve import *

In [2]:
# from netgen.csg import Pnt,SplineCurve2d,CSGeometry,Revolution,Sphere
import numpy as np
import netgen.meshing as ngm
from netgen.csg import *
from netgen.meshing import MeshingStep
from ngsolve.comp import IntegrationRuleSpaceSurface

In [3]:
from geometry import *

In [4]:
from ngsolve.webgui import Draw

In [5]:
from netgen.occ import SplineApproximation, Pnt, Axis, Face, Wire, Segment, Revolve, OCCGeometry, Z, X, Y
#%% Construction of Initial Curved Mesh
dim = 3
order = 1
msize = 0.15
dt = Parameter(0)

In [6]:
from ngsolve import Mesh

In [7]:
from esfem import Dziuk

### Dumbbell网格生成

In [8]:
maxh = 0.05
mesh, spline_obj = Mesh2dDumbbell(maxh,order,a=0.6,b=0.4,N_Spline=7,eps=2e-7)

tstart (must be vertically): (-3.59866e-07, 0.53507)
tend (must be vertically): (3.59865e-07, -0.53507)


### 长椭球网格生成

In [20]:
import numpy as np
from netgen.occ import Z

a, c = 1.0, 3.0
beta = 3.0  # 2~6 之间试

def CurveFunc(phi):
    s = np.sin(phi)
    z = c * np.tanh(beta*s)/np.tanh(beta)
    x = a*np.cos(phi)
    return (z, 0.0, x)

mesh = GetRotMesh(
    CurveFunc, msize=0.3,
    T_min=-np.pi/2, T_max=np.pi/2,
    axis=X, is_close=False, n=200
)

In [21]:
a, c = 1.0, 2.0  # 半轴长度：x,y方向=1，z方向=6
def CurveFunc(phi):
    return (c*np.sin(phi), 0.0, a*np.cos(phi))  # (x,0,z)

msize = 0.15   # 网格尺寸，自己调：越小越密
mesh = GetRotMesh(
    CurveFunc,
    msize=msize,
    T_min=-np.pi/2, T_max=np.pi/2,
    axis=X,
    is_close=False,
    n=300
)

In [22]:
mesh = GetTanhPinchedSpheroidMesh(msize = 0.3,c=2)

In [23]:
Draw(x,mesh,'xx')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [24]:
tauval = 1e-3
T_end = 0.46
Obj = Dziuk(mesh,T=T_end,tau=tauval)
t = Parameter(0)

mesh dim = 3
here


In [25]:
import ipywidgets as widgets
from IPython.display import display
import time
step = 0
import copy

In [26]:
Obj.IniByDisPos()
Obj.WeakMCF()

In [27]:
sceneu = Draw(Norm(Obj.X_old), mesh, 'disp')

t_old = 0.0
pbar = widgets.FloatProgress(
    value=0.0, min=0.0, max=1,
    description='t:', bar_style='info',
    layout=widgets.Layout(width='600px')
)
info = widgets.HTML(value="")
display(widgets.VBox([pbar, info]))

t0 = time.time()
while Obj.t<0.43:
    tauval = Obj.dt.Get()
    Obj.mesh.SetDeformation(Obj.Disp)
    Obj.lhs.Assemble()
    Obj.rhs.Assemble()
    Obj.Solution.vec.data = Obj.lhs.mat.Inverse(inverse="umfpack")*Obj.rhs.vec

    ## Solution.vec 代表的是X^m+1
    Obj.Disp.vec.data = BaseVector(Obj.Disp.vec.FV().NumPy() 
                                    + Obj.Solution.vec.FV().NumPy()
                                    - Obj.X_old.vec.FV().NumPy())
    Obj.X_old.vec.data = BaseVector(Obj.Solution.vec.FV().NumPy())
    Obj.t += tauval 
    sceneu.Redraw()
    
    # update progress UI
    pbar.value = min(Obj.t, 1)
    elapsed = time.time() - t0
    info.value = f"<b>step</b>={step} &nbsp; <b>t</b>={Obj.t:.6f}/{T_end} &nbsp; <b>elapsed</b>={elapsed:.1f}s"

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…